Import Tensorflow and open a session.

In [1]:
%run 'Ising.py'

### Determinant
Calculate matrix determinant

In [335]:
W = tf.constant([[0.,1.,1.,1.],
                 [-1.,0.,1.,-1.],
                 [-1.,-1.,0.,1.],
                 [-1.,1.,-1.,0.]])
f = tf.matrix_determinant(W)
sess.run(f)

9.0

Train by minimizing the ln(det(W)). TF knows how to do the gradient.

In [336]:
W = tf.Variable([[0.,1.,1.,1.],
                 [-1.,0.,1.,-1.],
                 [-1.,-1.,0.,1.],
                 [-1.,1.,-1.,0.]])
f = tf.log(tf.matrix_determinant(W))
init = tf.global_variables_initializer()
optimizer = tf.train.GradientDescentOptimizer(0.1)
train = optimizer.minimize(f)
sess.run(init)
for i in range(5):
    sess.run(train)
    print(sess.run(f))

2.06162
1.91632
1.75986
1.5904
1.4056


### Sparse Tensor
Sparse tensor must be converted to dense when passed to the determinant op.

In [337]:
W = tf.SparseTensor(indices=[[0, 0], [1, 1]], values=[1., 2.], dense_shape=[2, 2])
W_dense = tf.sparse_tensor_to_dense(W)
f = tf.log(tf.matrix_determinant(W_dense))
sess.run(f)

0.69314718

## Entanglement
### Entanglement Regions

In [43]:
list(Region({1,2},4).equivalences())

[{2, 3}, {0, 1}, {1, 2}, {0, 3}]

In [44]:
Region({1,2},4).config()

array([ 1.000, -1.000, -1.000,  1.000])

In [42]:
list(Regions(8).consecutives())

[set(),
 {0},
 {0, 1},
 {0, 1, 2},
 {0, 1, 2, 3},
 {0, 1, 2, 3, 4},
 {0, 1, 2, 3, 4, 5},
 {0, 1, 2, 3, 4, 5, 6},
 {0, 1, 2, 3, 4, 5, 6, 7}]

In [41]:
list(Regions(8).representatives())

[set(),
 {0},
 {0, 1},
 {0, 2},
 {0, 3},
 {0, 4},
 {0, 1, 2},
 {0, 1, 3},
 {0, 1, 4},
 {0, 2, 4},
 {0, 2, 5},
 {0, 1, 2, 3},
 {0, 1, 2, 4},
 {0, 1, 2, 5},
 {0, 1, 3, 5},
 {0, 1, 3, 6},
 {0, 1, 4, 5},
 {0, 2, 4, 6}]

### Entanglement Entropy
Entanglement entropy of free fermion.
$$H=\sum_{i}(1+(-)^i m)\mathrm{i}\chi_{i}\chi_{i+1}$$

In [40]:
system = FreeFermion(mass=1.e-10, size=8)
system.S([0,1,2,3,4])

0.84523244468672853

### Square Lattice Adjacency Matices

In [143]:
%run 'Ising.py'

In [144]:
machine = EFL(FreeFermion(mass=1.e-10, size=8), CylindricalModel(Lx = 4, Lz = 4))
writer = tf.summary.FileWriter('./log/test')
writer.add_graph(sess.graph)
sess.run(machine.model.initialize)

In [145]:
training_set = machine.training_set('representatives')
training_set 

{<tf.Tensor 'confs:0' shape=(?, 4) dtype=float64>: array([[ 1.000,  1.000,  1.000,  1.000],
        [-1.000,  1.000,  1.000,  1.000],
        [-1.000, -1.000,  1.000,  1.000],
        [-1.000,  1.000, -1.000,  1.000]]),
 <tf.Tensor 'S_sys:0' shape=(?,) dtype=float64>: array([ 0.000,  0.292,  0.304,  0.575])}

In [206]:
for i in range(5):
    sess.run(machine.model.train, training_set)
    print(sess.run(machine.model.cost, training_set))

0.0424190578727
0.0424190578665
0.0424190578603
0.0424190578542
0.042419057848


In [207]:
sess.run(machine.model.Smdl, training_set)

array([ 0.000,  0.302,  0.358,  0.503])

In [208]:
sess.run(machine.model.Jz, training_set)

array([ 0.290,  0.000,  0.000,  0.000])

In [209]:
sess.run(machine.model.Jx, training_set)

array([ 0.000,  0.000,  0.000])

In [210]:
sess.run(machine.model.h, training_set)

0.50764605894535575

In [191]:
machine.model.learning_rate.eval(session = sess)

0.01

In [193]:
machine.model.learning_rate.assign(0.02).eval(session = sess)

AttributeError: 'Tensor' object has no attribute 'assign'